## Overview

This notebook demonstrates the working of a Data Science Project using Apache Spark. The dataset was uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. The dataset has been taken from [Kaggle](https://www.kaggle.com/c/santander-customer-transaction-prediction/overview). 

The Project aims at predicting if a Santander customer will make a specific transaction in the future, irrespective of the amount of money transacted. The project will work through the data loading, exploratory analysis and feature selection, and finally building the model to correctly predict the behaviour.

## Loading the dataset

The dataset was uploaded to the DBFS. The following code reads the dataset from the filesystem. The parameters are provided for infering the schema, header and delimiter for the csv. We'll continue to work with the default datatypes provided by spark and avoid converting it into a pandas dataframe because the execution of toPandas() will bring the data to the driver node. And since it is a considerable size of dataset, we'll avoid doing this and work with the default datasets that come with Spark.

In [3]:
file_type = 'csv'
train_file_location = "/FileStore/tables/train.csv"
train = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .load(train_file_location)

In [4]:
test_file_location = "/FileStore/tables/test.csv"
test = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .load(test_file_location)

We'll now take a look at the dimensions of the training and test dataset.

In [6]:
print('training dataset shape - ', train.count(), 'x', len(train.columns), '\ntest dataset shape - ', test.count(), 'x', len(test.columns))

training dataset shape - 200000 x 202 
test dataset shape - 200000 x 201

In [7]:
print('Columns for the training dataset are: ', train.columns)

Columns for the training dataset are: ['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43', 'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50', 'var_51', 'var_52', 'var_53', 'var_54', 'var_55', 'var_56', 'var_57', 'var_58', 'var_59', 'var_60', 'var_61', 'var_62', 'var_63', 'var_64', 'var_65', 'var_66', 'var_67', 'var_68', 'var_69', 'var_70', 'var_71', 'var_72', 'var_73', 'var_74', 'var_75', 'var_76', 'var_77', 'var_78', 'var_79', 'var_80', 'var_81', 'var_82', 'var_83', 'var_84', 'var_85', 'var_86', 'var_87', 'var_88', 'var_89', 'var_90', 'var_91', 'var_92', 'var_93', 'var_94', 'var_95', 'var_96', 'var_97', 'var_98', 'var_99', 'var_100', 'var_101', 'var_102', 'var_103', 'var_104', 'var_105', 'var_106', 'var_107', 'var_108', 'var_109', 'var_110', 'var_111', 'var_112', 'var_113', 'var_114', 'var_115', 'var_116', 'var_117', 'var_118', 'var_119', 'var_120', 'var_121', 'var_122', 'var_123', 'var_124', 'var_125', 'var_126', 'var_127', 'var_128', 'var_129', 'var_130', 'var_131', 'var_132', 'var_133', 'var_134', 'var_135', 'var_136', 'var_137', 'var_138', 'var_139', 'var_140', 'var_141', 'var_142', 'var_143', 'var_144', 'var_145', 'var_146', 'var_147', 'var_148', 'var_149', 'var_150', 'var_151', 'var_152', 'var_153', 'var_154', 'var_155', 'var_156', 'var_157', 'var_158', 'var_159', 'var_160', 'var_161', 'var_162', 'var_163', 'var_164', 'var_165', 'var_166', 'var_167', 'var_168', 'var_169', 'var_170', 'var_171', 'var_172', 'var_173', 'var_174', 'var_175', 'var_176', 'var_177', 'var_178', 'var_179', 'var_180', 'var_181', 'var_182', 'var_183', 'var_184', 'var_185', 'var_186', 'var_187', 'var_188', 'var_189', 'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195', 'var_196', 'var_197', 'var_198', 'var_199']

In [8]:
print('Columns for the test dataset are - ', test.columns)

Columns for the test dataset are - ['ID_code', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43', 'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50', 'var_51', 'var_52', 'var_53', 'var_54', 'var_55', 'var_56', 'var_57', 'var_58', 'var_59', 'var_60', 'var_61', 'var_62', 'var_63', 'var_64', 'var_65', 'var_66', 'var_67', 'var_68', 'var_69', 'var_70', 'var_71', 'var_72', 'var_73', 'var_74', 'var_75', 'var_76', 'var_77', 'var_78', 'var_79', 'var_80', 'var_81', 'var_82', 'var_83', 'var_84', 'var_85', 'var_86', 'var_87', 'var_88', 'var_89', 'var_90', 'var_91', 'var_92', 'var_93', 'var_94', 'var_95', 'var_96', 'var_97', 'var_98', 'var_99', 'var_100', 'var_101', 'var_102', 'var_103', 'var_104', 'var_105', 'var_106', 'var_107', 'var_108', 'var_109', 'var_110', 'var_111', 'var_112', 'var_113', 'var_114', 'var_115', 'var_116', 'var_117', 'var_118', 'var_119', 'var_120', 'var_121', 'var_122', 'var_123', 'var_124', 'var_125', 'var_126', 'var_127', 'var_128', 'var_129', 'var_130', 'var_131', 'var_132', 'var_133', 'var_134', 'var_135', 'var_136', 'var_137', 'var_138', 'var_139', 'var_140', 'var_141', 'var_142', 'var_143', 'var_144', 'var_145', 'var_146', 'var_147', 'var_148', 'var_149', 'var_150', 'var_151', 'var_152', 'var_153', 'var_154', 'var_155', 'var_156', 'var_157', 'var_158', 'var_159', 'var_160', 'var_161', 'var_162', 'var_163', 'var_164', 'var_165', 'var_166', 'var_167', 'var_168', 'var_169', 'var_170', 'var_171', 'var_172', 'var_173', 'var_174', 'var_175', 'var_176', 'var_177', 'var_178', 'var_179', 'var_180', 'var_181', 'var_182', 'var_183', 'var_184', 'var_185', 'var_186', 'var_187', 'var_188', 'var_189', 'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195', 'var_196', 'var_197', 'var_198', 'var_199']

## Exploratory Analysis

We'll first start the exploratory analysis by visualizing the target variable and checking if the dataset is imbalanced or not. 

For doing this, we'll select the target function from the train dataframe after converting it into an RDD. We'll convert it into an RDD because we need to use the map function. In order to avoid heavy shuffiling across the cluster beacuse of the collect used, we'll sample the dataframe by the ratio of 10% using stratified sampling. This will lower our shuffling to a great extent and give us an idea about the dataset.

In [10]:
import seaborn as sns
target_values = train.select('target').sampleBy('target', {0:0.1, 1:0.1}).rdd.map(lambda x: x.target).collect()
display(sns.countplot(target_values))

The image shown above is for sampled dataset. We'll identify the actual ratio of the imabalance below.

In [12]:
train.select('target').groupby('target').count().toPandas()

,target,count
0,1,20098
1,0,179902


As we can see from the image, the dataset is highly imbalanced. Now we'll move on to do a check on the missing values in the dataset. We'll identify the number of missing values in the dataset. In order to do this we'll use the aggregated function. We'll build our own function null_check which takes the column name as parameter and returns the sum of null values back.

In [14]:
from pyspark.sql.functions import isnan, isnull, col
def null_check(c):
  vals = col(c).isNotNull() & isnan(c)
  return sum(vals.cast("integer")).alias(c)

train.agg(*[null_check(c) for c in train.columns]).first()

Out[79]: Row(ID_code=0, target=0, var_0=0, var_1=0, var_2=0, var_3=0, var_4=0, var_5=0, var_6=0, var_7=0, var_8=0, var_9=0, var_10=0, var_11=0, var_12=0, var_13=0, var_14=0, var_15=0, var_16=0, var_17=0, var_18=0, var_19=0, var_20=0, var_21=0, var_22=0, var_23=0, var_24=0, var_25=0, var_26=0, var_27=0, var_28=0, var_29=0, var_30=0, var_31=0, var_32=0, var_33=0, var_34=0, var_35=0, var_36=0, var_37=0, var_38=0, var_39=0, var_40=0, var_41=0, var_42=0, var_43=0, var_44=0, var_45=0, var_46=0, var_47=0, var_48=0, var_49=0, var_50=0, var_51=0, var_52=0, var_53=0, var_54=0, var_55=0, var_56=0, var_57=0, var_58=0, var_59=0, var_60=0, var_61=0, var_62=0, var_63=0, var_64=0, var_65=0, var_66=0, var_67=0, var_68=0, var_69=0, var_70=0, var_71=0, var_72=0, var_73=0, var_74=0, var_75=0, var_76=0, var_77=0, var_78=0, var_79=0, var_80=0, var_81=0, var_82=0, var_83=0, var_84=0, var_85=0, var_86=0, var_87=0, var_88=0, var_89=0, var_90=0, var_91=0, var_92=0, var_93=0, var_94=0, var_95=0, var_96=0, var_97=0, var_98=0, var_99=0, var_100=0, var_101=0, var_102=0, var_103=0, var_104=0, var_105=0, var_106=0, var_107=0, var_108=0, var_109=0, var_110=0, var_111=0, var_112=0, var_113=0, var_114=0, var_115=0, var_116=0, var_117=0, var_118=0, var_119=0, var_120=0, var_121=0, var_122=0, var_123=0, var_124=0, var_125=0, var_126=0, var_127=0, var_128=0, var_129=0, var_130=0, var_131=0, var_132=0, var_133=0, var_134=0, var_135=0, var_136=0, var_137=0, var_138=0, var_139=0, var_140=0, var_141=0, var_142=0, var_143=0, var_144=0, var_145=0, var_146=0, var_147=0, var_148=0, var_149=0, var_150=0, var_151=0, var_152=0, var_153=0, var_154=0, var_155=0, var_156=0, var_157=0, var_158=0, var_159=0, var_160=0, var_161=0, var_162=0, var_163=0, var_164=0, var_165=0, var_166=0, var_167=0, var_168=0, var_169=0, var_170=0, var_171=0, var_172=0, var_173=0, var_174=0, var_175=0, var_176=0, var_177=0, var_178=0, var_179=0, var_180=0, var_181=0, var_182=0, var_183=0, var_184=0, var_185=0, var_186=0, var_187=0, var_188=0, var_189=0, var_190=0, var_191=0, var_192=0, var_193=0, var_194=0, var_195=0, var_196=0, var_197=0, var_198=0, var_199=0)

Careful examination shows that no column in the training dataset has null values. We'll do the same with testing dataset.

In [16]:
test.agg(*[null_check(c) for c in test.columns]).first()

Out[80]: Row(ID_code=0, var_0=0, var_1=0, var_2=0, var_3=0, var_4=0, var_5=0, var_6=0, var_7=0, var_8=0, var_9=0, var_10=0, var_11=0, var_12=0, var_13=0, var_14=0, var_15=0, var_16=0, var_17=0, var_18=0, var_19=0, var_20=0, var_21=0, var_22=0, var_23=0, var_24=0, var_25=0, var_26=0, var_27=0, var_28=0, var_29=0, var_30=0, var_31=0, var_32=0, var_33=0, var_34=0, var_35=0, var_36=0, var_37=0, var_38=0, var_39=0, var_40=0, var_41=0, var_42=0, var_43=0, var_44=0, var_45=0, var_46=0, var_47=0, var_48=0, var_49=0, var_50=0, var_51=0, var_52=0, var_53=0, var_54=0, var_55=0, var_56=0, var_57=0, var_58=0, var_59=0, var_60=0, var_61=0, var_62=0, var_63=0, var_64=0, var_65=0, var_66=0, var_67=0, var_68=0, var_69=0, var_70=0, var_71=0, var_72=0, var_73=0, var_74=0, var_75=0, var_76=0, var_77=0, var_78=0, var_79=0, var_80=0, var_81=0, var_82=0, var_83=0, var_84=0, var_85=0, var_86=0, var_87=0, var_88=0, var_89=0, var_90=0, var_91=0, var_92=0, var_93=0, var_94=0, var_95=0, var_96=0, var_97=0, var_98=0, var_99=0, var_100=0, var_101=0, var_102=0, var_103=0, var_104=0, var_105=0, var_106=0, var_107=0, var_108=0, var_109=0, var_110=0, var_111=0, var_112=0, var_113=0, var_114=0, var_115=0, var_116=0, var_117=0, var_118=0, var_119=0, var_120=0, var_121=0, var_122=0, var_123=0, var_124=0, var_125=0, var_126=0, var_127=0, var_128=0, var_129=0, var_130=0, var_131=0, var_132=0, var_133=0, var_134=0, var_135=0, var_136=0, var_137=0, var_138=0, var_139=0, var_140=0, var_141=0, var_142=0, var_143=0, var_144=0, var_145=0, var_146=0, var_147=0, var_148=0, var_149=0, var_150=0, var_151=0, var_152=0, var_153=0, var_154=0, var_155=0, var_156=0, var_157=0, var_158=0, var_159=0, var_160=0, var_161=0, var_162=0, var_163=0, var_164=0, var_165=0, var_166=0, var_167=0, var_168=0, var_169=0, var_170=0, var_171=0, var_172=0, var_173=0, var_174=0, var_175=0, var_176=0, var_177=0, var_178=0, var_179=0, var_180=0, var_181=0, var_182=0, var_183=0, var_184=0, var_185=0, var_186=0, var_187=0, var_188=0, var_189=0, var_190=0, var_191=0, var_192=0, var_193=0, var_194=0, var_195=0, var_196=0, var_197=0, var_198=0, var_199=0)

Test dataset has no null values too. Now the next step going forward will be to examine the values in the dataset. Check if it has a skewness in it and any possible way to find out the collinearity in the columns.

In [18]:
sampled_train = train.stat.sampleBy('target', {0:0.1, 1:0.1})

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.figure()
columns = np.array([c for c in sampled_train.columns if c.startswith('var')]).reshape(40, 5)
fig, axes = plt.subplots(40, 5, squeeze=True, figsize=(30, 100))
for i, a in enumerate(axes):
  for j, s in enumerate(a):
    s.title.set_text('Dist of: %s'%columns[i, j])
    sns.distplot(sampled_train.select(columns[i, j]).rdd.flatMap(lambda x: x).collect(), ax=s)
  
display(fig)

We can see from the plots that the distribution of the columns is mostly normal, but a few of them are skewed. We'll check the skewness of the dataset and identify if something has to be done about it.

In [21]:
from pyspark.sql.functions import skewness
skew = list()
for c in train.columns:
  skew.append(train.select(skewness(train[c])).rdd.flatMap(lambda x: x).collect())

In [22]:
max(np.array(x[2:])), min(np.array(x[2:])), sum(np.array(x[2:]) > 0.5)

Out[32]: (array([-0.01399395]), array([-0.81181156]), array([0]))

The maximum skewness in the data is 0.26 which is relatively low. We'll check the kurtosis now

In [24]:
from pyspark.sql.functions import kurtosis
kurt = list()
for c in train.columns:
  kurt.append(train.select(kurtosis(train[c])).rdd.flatMap(lambda x: x).collect())

In [25]:
max(np.array(x[2:])), min(np.array(x[2:])), sum(np.array(x[2:]) > 0.5)

Out[33]: (array([-0.01399395]), array([-0.81181156]), array([0]))

## Feature Selection

We'll now try to analyse if we can use PCA and analyse if we can use the feature reduction technique here. In order to do that, we'll have to first do the scaling of the dataset as we do not know what scale the columns are in. After that we'll see how much variablity is being explained by the components. We'll use the sampled dataframe for this. We'll create a new sampled dataframe such that the target variable is equally balanced.

In [27]:
sampled_train = train.stat.sampleBy('target', {0:0.11, 1:1})

In [28]:
sampled_train = sampled_train.drop('ID_code')

Since the columns in our dataset have float values, we cannot directly apply the minmaxscaler to the dataset. We'll have to create a vector of the column and apply the minmaxscaler after that. The output will be original columns + scaled columns.

In [30]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA

columns = [c for c in sampled_train.columns if c != 'target']

assembler = VectorAssembler(inputCols=columns, outputCol='features')
scaler = MinMaxScaler(inputCol='features', outputCol='scaledfeatures')

pipeline = Pipeline(stages=[assembler, scaler])
sampled_train = pipeline.fit(sampled_train).transform(sampled_train)

In [31]:
pca = PCA(k=100, inputCol='scaledfeatures', outputCol='pcafeatures')
model = pca.fit(sampled_train)

In [32]:
sum(model.explainedVariance.values)

Out[7]: 0.5723450625434576

So even after taking 100 features from the PCA, we are still able to explain only 57% of the variability. So we'll drop the idea of taking PCA and focus on other methods to derive feature importance which will indeed help us in building model.

In [34]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="target", featuresCol="scaledfeatures",impurity='gini')
rf_model = rf.fit(sampled_train)
rf_model.featureImportances

Out[8]: SparseVector(200, {0: 0.0491, 1: 0.0043, 2: 0.0402, 3: 0.0005, 5: 0.0003, 6: 0.0434, 7: 0.0003, 8: 0.0003, 9: 0.0002, 10: 0.0005, 11: 0.0007, 12: 0.0518, 13: 0.0174, 14: 0.0003, 15: 0.0009, 18: 0.0057, 20: 0.0004, 21: 0.0187, 22: 0.023, 24: 0.0002, 25: 0.0008, 26: 0.0564, 27: 0.0003, 32: 0.0003, 33: 0.0041, 36: 0.0031, 38: 0.0002, 40: 0.0101, 44: 0.0295, 45: 0.0003, 48: 0.0012, 50: 0.0003, 53: 0.0011, 54: 0.0007, 55: 0.0003, 61: 0.0004, 64: 0.0004, 66: 0.002, 67: 0.0053, 68: 0.0006, 69: 0.0006, 70: 0.0008, 71: 0.0002, 75: 0.0009, 76: 0.017, 78: 0.0255, 79: 0.0003, 80: 0.0385, 81: 0.11, 82: 0.0001, 85: 0.0028, 86: 0.0015, 87: 0.0006, 88: 0.0006, 90: 0.0014, 91: 0.0067, 92: 0.0049, 93: 0.0004, 94: 0.0004, 95: 0.0003, 97: 0.0004, 99: 0.0118, 101: 0.0005, 108: 0.0012, 109: 0.0013, 110: 0.0571, 111: 0.0006, 113: 0.0003, 115: 0.0074, 118: 0.0009, 119: 0.0007, 122: 0.002, 123: 0.0056, 125: 0.0005, 126: 0.0001, 127: 0.0011, 128: 0.0003, 129: 0.0003, 130: 0.002, 131: 0.001, 132: 0.0003, 133: 0.0132, 134: 0.0007, 138: 0.0005, 139: 0.0749, 143: 0.0003, 144: 0.0003, 145: 0.0012, 146: 0.0319, 147: 0.0006, 148: 0.0068, 149: 0.0022, 150: 0.0008, 152: 0.0002, 154: 0.0009, 156: 0.0006, 157: 0.0003, 158: 0.0003, 159: 0.0008, 160: 0.0007, 162: 0.0005, 165: 0.0206, 166: 0.0449, 167: 0.0002, 169: 0.0108, 170: 0.0052, 171: 0.0004, 172: 0.0006, 173: 0.0061, 174: 0.0185, 178: 0.0002, 179: 0.0082, 181: 0.0003, 183: 0.0002, 184: 0.0073, 187: 0.0003, 190: 0.0231, 191: 0.0119, 192: 0.003, 193: 0.0009, 194: 0.0005, 198: 0.0143, 199: 0.0003})

In [35]:
import pandas as pd

feature_imp_df = pd.DataFrame({'colname':columns, 'imp':rf_model.featureImportances}).sort_values('imp', ascending=False)

In [36]:
feature_imp_df.head(25).T

,81,139,110,26,12,0,166,6,2,80,146,44,78,190,22,165,21,174,13,76,198,133,191,99,169
colname,var_81,var_139,var_110,var_26,var_12,var_0,var_166,var_6,var_2,var_80,var_146,var_44,var_78,var_190,var_22,var_165,var_21,var_174,var_13,var_76,var_198,var_133,var_191,var_99,var_169
imp,0.11002,0.0749469,0.0571374,0.0563614,0.0518402,0.0490933,0.0448938,0.0434187,0.0401545,0.0384796,0.0319045,0.0295492,0.0254687,0.0231182,0.022986,0.0205869,0.0186514,0.0185214,0.0174395,0.0169846,0.0143446,0.0131662,0.0119092,0.0117959,0.0108433


In [37]:
feature_imp_df[feature_imp_df['imp'] == 0.0].shape

So there are 58 columns in the dataset which contribute nothing towards the prediction of the model.

Now we'll select the top 25 important features and will use those going forward in order to do hyper-parameter tuning and create a final model.

In [40]:
imp_features = feature_imp_df['colname'][:25]
ind = feature_imp_df.index.values[:25]

Here we'll use the VectorSlicer to get the values from the features which we have scaled.

In [42]:
from pyspark.ml.feature import VectorSlicer

slicer = VectorSlicer(inputCol='scaledfeatures', outputCol='impfeatures', indices=ind)
sampled_train = slicer.transform(sampled_train)

## Splitting the data for training and validation.

Now, we have the set of important features and their values in the column 'impfeatures'. We'll now proceed to building various models.

In [44]:
X_train = train.sampleBy('target', {0:0.8, 1:0.8})
X_test = train.subtract(X_train)

In [45]:
X_train.groupby('target').count().toPandas()

,target,count
0,1,16134
1,0,143925


In [46]:
X_test.groupby('target').count().toPandas()

,target,count
0,1,3991
1,0,35948


## Transforming the training and validation data 

We'll create a function which takes the data in and transforms it into the format that we want using the pipeline we generated and slicer to get the values from only the important features.

In [48]:
def transformer(train, pipeline, slicer):
  df = spark.createDataFrame(sc.emptyRDD(), schema=train.schema)
  for i in range(5):
    df = df.union(train.sampleBy('target', {0:0.11, 1:1}))
  df = pipeline.fit(df).transform(df)
  df = slicer.transform(df)
  return df

In [49]:
def transformer_test(df, pipeline, slicer):
  df = pipeline.fit(df).transform(df)
  df = slicer.transform(df)
  return df

We'll transform the training data first to the required format.

In [51]:
X_train_1 = transformer(X_train, pipeline, slicer)

We'll transform the test data to the required format.

In [53]:
X_test = transformer_test(X_test, pipeline, slicer)

## Building  a model

Now we have the data in the required format. We'll proceed to building a most suitable model which gives us more accuracy towards the validation dataset. We'll also do some hyper-parameter tuning to get the desired results.

#### Logistic Regression

In [56]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='target')
lg = LogisticRegression(featuresCol='scaledfeatures', labelCol='target')

lg_model = lg.fit(X_train_1)
training_result_lr = lg_model.summary
predictions_lr = lg_model.transform(X_test)
print("Area Under ROC for test dataset: " + str(evaluator.evaluate(predictions_lr)))

Area Under ROC for test dataset: 0.8612999806721453

#### Random Forest Classifier

In [58]:
rf = RandomForestClassifier(labelCol="target", featuresCol="scaledfeatures", numTrees=50)
rf_model = rf.fit(X_train_1)
#training_result_rf = rf_model.summary
predictions_rf = rf_model.transform(X_test)
print("Area Under ROC for test dataset: " + str(evaluator.evaluate(predictions_rf)))

Area Under ROC for test dataset: 0.7420819745612224

#### GBTClassifier

The classfier gave best performance at stepsize 0.45. The performance deteiorated when used other hyper-parameters.

In [60]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="target", featuresCol="scaledfeatures", stepSize=0.5)
gbt_model = gbt.fit(X_train_1)
predictions_gbt = gbt_model.transform(X_test)
print("Area Under ROC for test dataset: " + str(evaluator.evaluate(predictions_gbt)))

Area Under ROC for test dataset: 0.8007288376610829

#### Multi Layer Perceptron Classifier

In [62]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [25, 20, 10, 2]

mp = MultiLayerPerceptronClassifier(maxIters=10, layers=layers, blocksize=128)
mp_model = mp.fit(X_train_1)
predictions_mp = mp_model.transform(X_test)
print("Area Under ROC for test dataset: " + str(evaluator.evaluate(predictions_mp)))

In [63]:
import matplotlib.pyplot as plt
plt.figure(figsize=(3,3))
roc = training_result.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
display(plt.show())

In [64]:
predictions_lr.select('target', 'prediction').groupby(['target', 'prediction']).count().toPandas()

,target,prediction,count
0,1,0.0,722
1,0,0.0,117433
2,1,1.0,3387
3,0,1.0,46808
